In [ ]:
import altair as alt
#from altair import pipe, limit_rows, to_values
import altair_viewer
'''t = lambda data: pipe(data, limit_rows(max_rows=10000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')'''
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

In [ ]:
import pandas as pd
from itertools import chain
import altair as alt

from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=10000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

Di1 = 'Region'
Di2 = 'State'
Measure = 'Quantity'
'''df = pd.read_csv('Superstore.csv', encoding='windows-1252')
ValDi1 = list(chain.from_iterable(csvManager.getAxisYName([Di1])))
ValDi2 =  list(chain.from_iterable(csvManager.getAxisYName([Di2])))
ValMes = csvManager.getDataForBar([Di2],[Measure])'''
#print(ValDi1,ValDi2,ValMes)
f1 = 'SS_20lines.csv'
f2 = 'SS_100lines.csv'
f3 = 'Superstore.csv'
df = pd.read_csv(f3, encoding='windows-1252')
c = alt.Chart(df).mark_bar().encode(
    x=str(Di2+':N'),
    y=str(Measure+':Q')
).resolve_scale(
    x = 'independent'
)



In [ ]:
import altair as alt
import pandas as pd

Di1 = 'Country/Region'
Di2 = 'Region'
Di3 = 'State'
Measure = 'Quantity'

#f1 = 'SS_20lines.csv'
#f2 = 'SS_100lines.csv'
f3 = 'Superstore.csv'
df = pd.read_csv(f3, encoding='windows-1252')

c = alt.Chart(df).mark_bar().encode(
    x=str(Di3+':N'),
    y=('median(Quantity):Q'),
    color=str(Di2+':N')
).facet(column=str(Di2+':N')
).resolve_scale(x = 'independent')



In [ ]:
c = alt.Chart(df).mark_bar().encode(
    x=str('State:N'),
    y=('median(Quantity):Q'),
    color=str('Region:N')
).facet(column=str('Region:N')
).resolve_scale(x = 'independent')


In [ ]:
import altair as alt
import pandas as pd

Di1 = 'Country/Region'
Di2 = 'Region'
Di3 = 'State'
Measure = 'Quantity'

f1 = 'SS_20lines.csv'
f2 = 'SS_100lines.csv'
f3 = 'Superstore.csv'
df = pd.read_csv(f3, encoding='windows-1252')

c = alt.Chart(df).mark_bar().encode(
    y=str(Di2+':N'),
    x=str(Measure+':Q'),
    color=str('month(Order Date):T')
).facet(row=str('month(Order Date):T')
).resolve_scale(y = 'independent')



In [ ]:
import pandas as pd
import altair as alt


Di = 'Sub-Category'
Meas = 'Quantity'
f3 = 'Superstore.csv'

df = pd.read_csv(f3, encoding='windows-1252')
df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')

base = alt.Chart(df).encode(
    theta=alt.Theta('sum(Quantity):Q'), color=alt.Color('year(Order Date):T', type="nominal"), tooltip = 'sum(Quantity):Q'
)

pie = base.mark_arc(outerRadius=120)
#text = base.mark_text(radius=140, size=20).encode(text=str(Di+':N'))

 #+ text

In [ ]:
import pandas as pd

f3 = 'Superstore.csv'
df = pd.read_csv(f3, encoding='windows-1252')

df ['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Order year'] = df['Order Date'].dt.year
df['Order month'] = df['Order Date'].dt.month
df['Order day'] = df['Order Date'].dt.day
df['Order day'] = df['Order Date'].dt.day
df['Order month'] = df['Order Date'].dt.month
df.columns

In [ ]:
import altair as alt
import pandas as pd
import altair_viewer
#import csvManager
'''from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=10000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')'''
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

Measure = ['Quantity','Profit','Discount','Sales']



def plotBar(row,column,f):      #string == Measure
    fil = f            ###sum
    fd = fd2           ###month
    if len(column) == 1 or len(row) == 1:     #column 1 , row 1
        if type(row) == type('string'):               #Measure in Row
            sy = str(fil+'('+row+')')
        else:
            if df[row[0]].dtypes == 'datetime64[ns]':   #Dimension(Datetime) in Row
                sy = str(fd+'('+row[0]+')')
            else:
                sy = str(row[0])                        #normal Dimension in Row

        if type(column) == type('string'):           #Measure in Column
            sx = str(fil+'('+column+')')
        else:
            if df[column[0]].dtypes == 'datetime64[ns]':    #Dimension(Datetime) in column
                sx = str(fd+'('+column[0]+')')
            else:
                sx = str(column[0])                     #normal Dimension in Column
        
        c = alt.Chart(df).mark_bar().encode(
            x=sx,
            y=sy,
            tooltip = [sy,sx]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
    else:
                                           #2 Dimension
        if type(row) == type('string') :               #2 Column (Dimension)
            Di = column
            Me = row
            if df[Di[-2]].dtypes == 'datetime64[ns]':
                scol = str(fd+'('+Di[-2]+')')           
                sx = str(Di[-1])
            elif df[Di[-1]].dtypes == 'datetime64[ns]':
                scol = str(Di[-2])
                sx = str(fd+'('+Di[-1]+')')
            else:
                scol = str(Di[-2])                      #column > x
                sx = str(Di[-1])

            c = alt.Chart(df).mark_bar().encode(
                x=sx,
                y=str(fil+'('+Me+')'),
                color=scol,
                tooltip = [sx,str(fil+'('+Me+')')]
            ).facet(column=scol
            ).resolve_scale(x = 'independent')
            return c
        elif type(column) == type('string'):                 #2 Row (Dimension)
            Di = row    
            Me = column
            if df[Di[-2]].dtypes == 'datetime64[ns]':            #year,date error (large data)
                srow = str(fd+'('+Di[-2]+')')
                sy = str(Di[-1])

            elif df[Di[-1]].dtypes == 'datetime64[ns]':
                srow = str(Di[-2])
                sy = str(fd+'('+Di[-1]+')')
            else:
                srow = str(Di[-2])
                sy = str(Di[-1])
            c = alt.Chart(df).mark_bar().encode(
                x=str(fil+'('+Me+')'),
                y=sy,
                color=srow,
                tooltip = [sy,str(fil+'('+Me+')')]
            ).facet(row=srow
            ).resolve_scale(y = 'independent')
            return c
        else:
            return #'Pls enter 1 or 2 Dimension'
        
def plotLine(row,column,f):

    fil = f                                        ###sum
    fd = fd2                                    ###month
    if type(column) == type('string'):                 #column is Measure
        if len(row) == 1:
            #if df[row[0]].dtypes == 'datetime64[ns]':
            Di = row[0]
            Me = column
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q')),
                alt.Y(str(fd+'('+Di+'):T')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        else:
            Me = column
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q')),
                alt.Y(
                    alt.repeat('layer'),timeUnit = fil,type='temporal',
                ),
                color = alt.datum(alt.repeat('layer')),
                tooltip = [str(fil+'('+Me+')')]
            ).repeat(layer=[row[0],row[1]])
            return ch
    else:   #row is Measure

        #elif df[c].dtypes == 'datetime64[ns]':
        if len(column) == 1:
            Me = row
            Di = column[0]
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fd+'('+Di+'):T')),
                alt.Y(str(fil+'('+Me+'):Q')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        else:
            Me = row
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(
                    alt.repeat('layer'),timeUnit = fil,type='temporal',
                ),
                alt.Y(str(fil+'('+Me+'):Q'),
                color = alt.datum(alt.repeat('layer')),
                tooltip = [str(fil+'('+Me+')')]
            ).repeat(layer=[column[0],column[1]])
            return ch

def plotPie(row,column,f):

    fil = f                                       ####sum
    fd = fd3                                     ####year
    if row[0] in Measure:
        Mes = row[0]
        Di = column[0]
    elif column[0] in Measure:
        Mes = column[0]
        Di = row[0]

    if df[Di].dtypes == 'datetime64[ns]':
        s = str(fd+'('+Di+'):T')
    else:
        s = str(Di+':N')

    base = alt.Chart(df).mark_arc().encode(
        theta=alt.Theta(str(fil+'('+Mes+'):Q')), 
        color=alt.Color(s, type="nominal"), 
        tooltip = [s,str(fil+'('+Mes+'):Q')]
    )

    return base

def plotGraph(row,column,fil,tp):

    def checkMeasure(R,C):      #True when row is measure
        for r in R:
            if r in Measure:
                return True
            else:
                return False
    f = fil

    if tp == 'line':
        if checkMeasure(row,column):    #row is measure
            if len(row) == 1:
                return alt.vconcat(plotLine(row[0],column,f))
            elif len(row) == 2:
                return alt.vconcat(plotLine(row[0],column,f),plotLine(row[1],column,f))
            elif len(row) == 3:
                return alt.vconcat(plotLine(row[0],column,f),plotLine(row[1],column,f),plotLine(row[2],column,f))
            elif len(row) == 4:
                return alt.vconcat(plotLine(row[0],column,f),plotLine(row[1],column,f),plotLine(row[2],column,f),plotLine(row[3],column,f))
        else:
            if len(column) == 1:
                return alt.hconcat(plotLine(row,column[0],f))
            elif len(column) == 2:
                return alt.hconcat(plotLine(row,column[0],f),plotLine(row,column[1],f))
            elif len(column) == 3:
                return alt.hconcat(plotLine(row,column[0],f),plotLine(row,column[1],f),plotLine(row,column[2],f))
            elif len(column) == 4:
                return alt.hconcat(plotLine(row,column[0],f),plotLine(row,column[1],f),plotLine(row,column[2],f),plotLine(row,column[3],f))
    elif tp == 'bar':
        if checkMeasure(row,column):    #row is measure
            if len(row) == 1:
                return alt.vconcat(plotBar(row[0],column,f))
            elif len(row) == 2:
                return alt.vconcat(plotBar(row[0],column,f),plotBar(row[1],column,f))
            elif len(row) == 3:
                return alt.vconcat(plotBar(row[0],column,f),plotBar(row[1],column,f),plotBar(row[2],column,f))
            elif len(row) == 4:
                return alt.vconcat(plotBar(row[0],column,f),plotBar(row[1],column,f),plotBar(row[2],column,f),plotBar(row[3],column,f))
        else:
            if len(column) == 1:
                return alt.hconcat(plotBar(row,column[0],f))
            elif len(column) == 2:
                return alt.hconcat(plotBar(row,column[0],f),plotBar(row,column[1],f))
            elif len(column) == 3:
                return alt.hconcat(plotBar(row,column[0],f),plotBar(row,column[1],f),plotBar(row,column[2],f))
            elif len(column) == 4:
                return alt.hconcat(plotBar(row,column[0],f),plotBar(row,column[1],f),plotBar(row,column[2],f),plotBar(row,column[3],f))

    elif tp == 'pie':
        plotPie(row,column,f)


df = pd.read_csv('Superstore.csv', encoding='windows-1252')
df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')

fd1 = 'date'
fd2 = 'month'
fd3 = 'year'

D1 = 'Order Date'
D2 = 'Ship Date'

D3 = 'Category'
D4 = 'Sub-Category'

D5 = 'Country/Region'
D6 = 'Region'
D7 = 'State'
D8 = 'City'

D9 = 'Ship Mode'

M1 = 'Quantity'
M2 = 'Discount'
M3 = 'Profit'
M4 = 'Sales'

f1 = 'sum'
f2 = 'mean'
f3 = 'max'
f4 = 'min'
f5 = 'average'  #    == mean
f6 = 'count' 

row = [D2]
column = [M1,M2,M3,M4]

column , row = row ,column

#if len(row)  1:

#plotBar(row,column,f1)
#plotLine(row,column,f1)
#plotPie(row,column,f1)
plotGraph(row,column,f1,'line')
    

In [ ]:
print(type('1')==type('eiei'))

In [ ]:
import altair as alt
from vega_datasets import data
import pandas as pd


df = pd.read_csv('Superstore.csv', encoding='windows-1252')

df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')

c = alt.Chart(df).mark_bar().encode(
    alt.X(alt.repeat("column"), aggregate = 'sum',type='quantitative'),
    alt.Y(alt.repeat("row"),type='nominal'),
    tooltip = [alt.Tooltip(alt.repeat("column"),aggregate = 'sum',type='quantitative'),
            alt.Tooltip(alt.repeat("row"),type='nominal')]
).repeat(
    row=['Region'],
    column=['Sales', 'Profit']
).interactive().resolve_scale(
    x = 'independent',
    y = 'independent'
).facet(
    column = 'Category'
)

c

In [ ]:
bar1 = alt.Chart(df).mark_bar().encode(
    y = alt.Y('sum(Profit)'),
    x = alt.X('Sub-Category', title=None),
    tooltip=['sum(Profit)','Sub-Category'],
).facet(column='Category').resolve_scale(
  x='independent'
)

bar2 = alt.Chart(df).mark_bar().encode(
    y = alt.Y('sum(Sales)'),
    x = alt.X('month(Order Date)', title=None),
    tooltip=['sum(Sales)','month(Order Date)'],
).resolve_scale(
  x='independent'
)

bar = alt.vconcat(bar1,bar1)
bar

In [ ]:
import altair as alt
from vega_datasets import data


source = data.cars()

# Brush for selection
brush = alt.selection(type='single', encodings=['x'])

# Histogram base
hist_base = alt.Chart(source).mark_bar(color='grey').encode(
    x=alt.X('Horsepower:Q', bin=True),
    y='count()',
).add_selection(brush)

# Histogram selection
hist_selection = alt.Chart(source).mark_bar().encode(
    x=alt.X('Horsepower:Q', bin=True),
    y='count()',
).transform_filter(brush)

# Base chart for data tables
ranked_text = alt.Chart(source).mark_text(align='right').encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<16
)

# Data Tables
horsepower = ranked_text.encode(text='Horsepower:N').properties(title=alt.TitleParams(text='Horsepower', align='right'))
mpg = ranked_text.encode(text='Miles_per_Gallon:N').properties(title=alt.TitleParams(text='MPG', align='right'))
origin = ranked_text.encode(text='Origin:N').properties(title=alt.TitleParams(text='Origin', align='right'))
text = alt.hconcat(horsepower, mpg, origin) # Combine data tables

# Build chart
alt.hconcat(
    hist_base+hist_selection,
    text
).resolve_legend(
    color="independent"
).configure_view(strokeWidth=0)

In [ ]:
import pandas as pd
import json

df = pd.read_csv('SS_20lines.csv', encoding='windows-1252')
data = {'dataframe':df,'row':[],'column':[],'typegraph':''}
def createjson() :
    data_json = json.dumps(data)
    jsonFile = open("data.json","w")
    jsonFile.write(data_json)
    jsonFile.close()
    with open ('data.json', encoding = 'utf8' ) as data_json :
        data_read = data_json.read()
    data_read 

js = df.to_json(orient = 'columns')
df2 = pd.read_json(js)
#createjson()

In [27]:
import altair as alt
from vega_datasets import data
import pandas as pd


df = pd.read_csv('Superstore.csv', encoding='windows-1252')

df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')

ch = alt.Chart(df).mark_line(point=True).encode(
    alt.X(
        alt.repeat("layer"),timeUnit= 'month',type='temporal'
    ),
    alt.Y('sum(Profit)'),
    color = alt.datum(alt.repeat('layer')),
    tooltip = ['sum(Profit)']
).repeat(layer=['Ship Date','Order Date'])
ch


alt.RepeatChart(...)